**CLASSIFIER CHAINS with TF-IDF**

In [ ]:
!pip install scikit-learn==0.24.1
!pip install scikit-multilearn

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from skmultilearn.adapt import MLkNN
from sklearn.metrics import hamming_loss, accuracy_score, classification_report

from google.colab import drive
drive.mount('/content/drive')
print('mount success')

aspects_df = pd.read_csv('/content/drive/Shareddrives/EMOThreat/train_set_taskA.csv')
X = aspects_df["Sentences"]
y = np.asarray(aspects_df[aspects_df.columns[:7]])
  
# initializing TfidfVectorizer 
vetorizar = TfidfVectorizer(max_features=7000, max_df=0.85)
# fitting the tf-idf on the given data
vetorizar.fit(X)
  
# splitting the data to training and testing data set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)
  
# transforming the data
X_train_tfidf = vetorizar.transform(X_train)
X_test_tfidf = vetorizar.transform(X_test)

# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.linear_model import LogisticRegression
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(LogisticRegression())
# Training logistic regression model on train data
classifier.fit(X_train_tfidf, y_train)
# predict
predictions = classifier.predict(X_test_tfidf)
# accuracy
print("\nReport: classifier chains")
print("Accuracy = ",accuracy_score(y_test,predictions))
print(classification_report(y_test,predictions))

Test with the model

In [ ]:
import pandas as pd
aspects_df = pd.read_csv('/content/drive/Shareddrives/EMOThreat/test_set_taskA.csv')
X = aspects_df["Sentences"]
vetorizar = TfidfVectorizer(max_features=7000, max_df=0.85)
# fitting the tf-idf on the given data
vetorizar.fit(X)
X = vetorizar.transform(X)
pred=classifier.predict(X)
pred=pred.astype(int)
csv = pd.DataFrame(pred.toarray(),columns=["anger","disgust","fear","sadness","surprise","happiness","neutral"])
csv.insert(7,"Sentences",aspects_df["Sentences"],True)
csv.to_csv("pred_tfidf_cc.csv", index=False)